# Table of Contents
 <p><div class="lev1 toc-item"><a href="#reading-and-preparing-data" data-toc-modified-id="reading-and-preparing-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>reading and preparing data</a></div>

# reading and preparing data

In [3]:
#library(ggbiplot)
library(tidyverse, quiet=T)
library(traj)
library(caret, quiet=T)

In [4]:


dat0 <- read.csv("../training_ultrasound.csv")

# Use Hadlok equation to estimate fetal weight from the 4 key ultrasound measurements
dat.raw = dat0 %>%
    mutate(
        LOG10.FWT.GM = 1.3596 + 0.0064*HCIRCM + 0.0424*ABCIRCM + 0.174*FEMURCM + 0.00061*BPDCM*ABCIRCM - 0.00386*ABCIRCM*FEMURCM,
        WTKG.estimate = ifelse(AGEDAYS<1 ,(10^LOG10.FWT.GM)/1000 ,WTKG) ,
        Study = paste('Study', STUDYID)
        )
dat.raw %>% glimpse
# clean data, remove samples with only one observation and after-birth
dat = dat.raw %>% group_by(SUBJID) %>%
    mutate(tot.measurements = n()) %>%
    filter(tot.measurements >1) %>%
    filter(AGEDAYS<0)


dat.train.raw = dat %>% ungroup %>%
    select(GAGEDAYS, SUBJID, ABCIRCM, HCIRCM, BPDCM, FEMURCM, BWT_40) %>%
    filter(complete.cases(.)) %>% group_by(SUBJID) %>%
    mutate(measurementId = row_number(), tot_measurements=n()) %>%
#    filter(measurementId < 7) %>%
#    filter(tot_measurements >1) %>% 
    ungroup
#     filter(GAGEDAYS > 150, GAGEDAYS <210)

dat.train = dat.train.raw

dat.train.raw %>% head

Observations: 17,370
Variables: 37
$ STUDYID       <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...
$ SUBJID        <int> 1002, 1002, 1002, 1003, 1003, 1003, 1003, 1003, 1003,...
$ SEXN          <int> 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...
$ SEX           <fctr> Female, Female, Female, Male, Male, Male, Male, Male...
$ GAGEBRTH      <int> 276, 276, 276, 280, 280, 280, 280, 280, 280, 280, 280...
$ BIRTHWT       <int> 3540, 3540, 3540, 3100, 3100, 3100, 3100, 3100, 3100,...
$ BIRTHLEN      <dbl> 50.3, 50.3, 50.3, 50.3, 50.3, 50.3, 50.3, 50.3, 50.3,...
$ BIRTHHC       <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N...
$ DELIVERY      <fctr> Category 2.0, Category 2.0, Category 2.0, Category 2...
$ PARITY        <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...
$ GRAVIDA       <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...
$ GAGEDAYS      <int> 255, 277, 669, 223, 224, 263, 266, 281, 321, 463, 645...
$ AGEDAYS       <

GAGEDAYS,SUBJID,ABCIRCM,HCIRCM,BPDCM,FEMURCM,BWT_40,measurementId,tot_measurements
255,1002,32.5,33.4,9.1,7.3,3.614882,1,1
223,1003,25.6,29.2,8.2,6.3,3.100000,1,4
224,1003,25.6,29.2,8.2,6.3,3.100000,2,4
263,1003,30.2,31.0,9.1,7.2,3.100000,3,4
266,1003,30.2,31.0,9.1,7.2,3.100000,4,4
169,1005,18.7,21.9,5.9,4.1,2.988224,1,3


In [6]:
dat.gbm = train(BWT_40~., data=dat.train.raw, method="gbm", preProcess=c("center", "scale", "knnImpute"), verbose=F)


Loading required package: gbm
Loading required package: survival

Attaching package: ‘survival’

The following object is masked from ‘package:caret’:

    cluster

Loading required package: splines
Loading required package: parallel
Loaded gbm 2.1.1
Loading required package: plyr
------------------------------------------------------------------------------
You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)
------------------------------------------------------------------------------

Attaching package: ‘plyr’

The following objects are masked from ‘package:dplyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following object is masked from ‘package:purrr’:

    compact



In [7]:
varImp(dat.gbm)

gbm variable importance

                 Overall
ABCIRCM          100.000
GAGEDAYS          71.652
SUBJID            17.379
BPDCM             15.618
FEMURCM           11.158
HCIRCM             8.963
tot_measurements   4.792
measurementId      0.000

In [10]:
dat.gbm = train(BWT_40~., data=dat.train.raw, method="gbm", preProcess=c("center", "scale", "knnImpute"),  weight=GAGEDAYS, verbose=F)


Warning message in eval(expr, envir, enclos):
“model fit failed for Resample01: shrinkage=0.1, interaction.depth=1, n.minobsinnode=10, n.trees=150 Error in method$fit(x = x, y = y, wts = wts, param = tuneValue, lev = obsLevels,  : 
  object 'GAGEDAYS' not found
”Warning message in method$fit(x = x, y = y, wts = wts, param = tuneValue, lev = obsLevels, :
“restarting interrupted promise evaluation”Warning message in method$fit(x = x, y = y, wts = wts, param = tuneValue, lev = obsLevels, :
“restarting interrupted promise evaluation”Warning message in method$fit(x = x, y = y, wts = wts, param = tuneValue, lev = obsLevels, :
“restarting interrupted promise evaluation”Warning message in eval(expr, envir, enclos):
“model fit failed for Resample01: shrinkage=0.1, interaction.depth=2, n.minobsinnode=10, n.trees=150 Error in method$fit(x = x, y = y, wts = wts, param = tuneValue, lev = obsLevels,  : 
  object 'GAGEDAYS' not found
”Warning message in method$fit(x = x, y = y, wts = wts, param = tun

Something is wrong; all the RMSE metric values are missing:
      RMSE        Rsquared  
 Min.   : NA   Min.   : NA  
 1st Qu.: NA   1st Qu.: NA  
 Median : NA   Median : NA  
 Mean   :NaN   Mean   :NaN  
 3rd Qu.: NA   3rd Qu.: NA  
 Max.   : NA   Max.   : NA  
 NA's   :9     NA's   :9    


ERROR: Error in train.default(x, y, weights = w, ...): Stopping
